# Function & Closures

# Function pointer

* A function pointer in Rust is a type that represents a reference to a function: `fn (arguments) -> return_type`. 

* Assigning a function to a variable creates a function pointer but requires exploicit coercion to a `fn` type.



In [11]:
fn add(a: i32, b: i32) -> i32 {
    a + b
}

fn multiply(a: i32, b: i32) -> i32 {
    a * b
}

let mut math_op: fn(i32, i32) -> i32 = add; // explicit coercion to function pointer type is required

println!("5 + 3 = {}", math_op(5, 3));

math_op = multiply;

println!("5 * 3 = {}", math_op(5, 3));

// `fn` implements `std::fmt::Pointer` trait
println!("Function pointer address: {:p}", math_op);

5 + 3 = 8
5 * 3 = 15
Function pointer address: 0x7ffd89661200


* Passing function pointers as arguments to other functions is straightforward, allowing for higher-order functions and callbacks.

In [13]:
fn update_all(values: &mut [i32], operation: fn(i32) -> i32) {
    for value in values.iter_mut() {
        *value = operation(*value);
    }
}

fn square(x: i32) -> i32 {
    x * x
}

let mut numbers = vec![1, 2, 3, 4, 5];
update_all(&mut numbers, square);
println!("Squared numbers: {:?}", numbers);

Squared numbers: [1, 4, 9, 16, 25]


## Closures

* Closures are anonymous functions that can capture variables from their surrounding scope. They are defined using vertical bars `|` to enclose parameters, followed by an expression or block of code.
* By default, closures infer the types of their parameters and return values, making them more flexible than regular functions.
* Closures can capture variables from their environment in three ways: 
  * by borrowing immutably, 
  * by borrowing mutably, 
  * or by taking ownership.

### Fn

* The `Fn` trait is used for closures that capture variables by borrowing them immutably. These closures can be called multiple times without mutating the captured variables.

In [22]:
fn update_all<F>(values: &mut [i32], operation: F)
where
    F: Fn(i32) -> i32,
{
    for value in values.iter_mut() {
        *value = operation(*value);
    }
}

fn main() {
    let factor = 10;
    let multiply_by_factor = |x: i32| x * factor; // closure capturing (borrowing immutably) `factor`

    assert!(multiply_by_factor(3) == 30);

    let mut numbers = vec![1, 2, 3, 4, 5];
    update_all(&mut numbers, multiply_by_factor);
    println!("Numbers multiplied by factor: {:?}", numbers);
}

main();

Numbers multiplied by factor: [10, 20, 30, 40, 50]


### FnMut

* The `FnMut` trait is for closures that capture variables by borrowing them mutably. These closures can modify the captured variables and can also be called multiple times.

In [23]:
fn for_each(values: &[i32], mut operation: impl FnMut(i32)) {
    for &value in values {
        operation(value);
    }
}

fn main() {
    let mut counter = 0;

    let mut even_counter = |n| { // closure captures `counter` mutably
        if n % 2 == 0 {
            counter += 1;
        }
    };

    let numbers = (0..10).collect::<Vec<i32>>();

    for_each(&numbers, &mut even_counter);
    
    println!("Count of even numbers: {}", counter);
}

main();

Count of even numbers: 5


### FnOnce

* The `FnOnce` trait is for closures that capture variables by taking ownership. These closures can only be called once, as they consume the captured variables when invoked.

In [2]:
use std::collections::HashMap;

fn main() {
    let dict_en_pl: HashMap<&str, &str> = 
        [("one", "jeden"), ("two", "dwa"), ("three", "trzy")]
        .iter().cloned().collect();

    let translate = move |word: &str| { // closure captures `name` by taking ownership
        if let Some(translation) = dict_en_pl.get(word) {
            println!("The Polish word for '{}' is '{}'", word, translation);
        } else {
            println!("No translation found for '{}'", word);
        }
    };

    translate("one");
    translate("four");

    //println!("Dictionary used for translation: {:?}", dict_en_pl);
}

main();

The Polish word for 'one' is 'jeden'
No translation found for 'four'
